### Урок 2

#### Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

*Наименование вакансии
*Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
*Ссылку на саму вакансию        
*Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [ ]:
def _parser_superjob(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': vacancy
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }
    
    link = 'https://www.superjob.ru/vacancy/search/'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
    
        page_block = parsed_html.find('a', {'class': 'f-test-button-1'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            vacancy_items = parsed_html.find_all('div', {'class': 'f-test-vacancy-item'})
                        
            for item in vacancy_items:
                vacancy_date.append(_parser_item_superjob(item))
                
    return vacancy_date

In [ ]:
def _parser_item_superjob(item):

    vacancy_date = {}
    
    # vacancy_name
    vacancy_name = item.find_all('a')
    if len(vacancy_name) > 1:
        vacancy_name = vacancy_name[-2].getText()
    else:
        vacancy_name = vacancy_name[0].getText()
    vacancy_date['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.find('span', {'class': 'f-test-text-vacancy-item-company-name'})
    
    if not company_name:
        company_name = item.findParent() \
                            .find('span', {'class': 'f-test-text-vacancy-item-company-name'}) \
                            .getText()
    else:
        company_name = company_name.getText()
    
    vacancy_date['company_name'] = company_name
    
    #salary
    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                  .findChildren()
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_currency = salary[-1].getText()
        salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                                .getText() \
                                .replace(u'\xa0', u' ') \
                                .split(' ', 1)[0]
        if salary == 'до' or len(salary) == 2:
            salary_min = None
            salary_max = None
        elif salary == 'от':
            salary_min = None
            salary_max = None
        else:
            salary_min = None
            salary_max = None           
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    
    # link
    vacancy_link = item.find_all('a')
    
    if len(vacancy_link) > 1:
        vacancy_link = vacancy_link[-2]['href']
    else:
        vacancy_link = vacancy_link[0]['href']
    
    vacancy_date['vacancy_link'] = f'https://www.superjob.ru{vacancy_link }'
    
    # site
    vacancy_date['site'] = 'www.superjob.ru'
    
    return vacancy_date

In [ ]:
def parser_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(_parser_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)

    return df

In [ ]:
vacancy = 'Python'
df = parser_vacancy(vacancy)

In [ ]:
df[0:100]